In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import numpy as np
from matplotlib import pyplot as plt

import os
from PIL import Image

data1_path = 'RaceHorses_416x240_30' # 26 x 15

In [19]:
df =[]
n_images = len(os.listdir(data1_path))
for im_name in os.listdir(data1_path):
    input_im =  np.array(Image.open(data1_path + '/' + im_name))[...,:3]
    df.append(input_im)
df = np.array(df)

(300, 240, 416, 3)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(64,(7,7),activation='relu',input_shape=(240, 416, 3)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(32,(5,5),acitvation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(16,(1,1),acitvation='relu'))
model.add(layers.Conv2D(8,(3,3),acitvation='relu'))
model.add(layers.Conv2D(4,(3,3),acitvation='relu'))